1) Необходимо прочитать файл cinema.json, используя библиотеку json. https://docs.python.org/3/library/json.html

Создать новые файлы cinema2.json и cinema2.csv, в которых для каждого кинотеатра будут отображены только следующие поля из исходного файла: 

* ID, 
* Краткое наименование (CommonName), 
* Адрес (текстовой строкой, только улица-дом, без индекса и района), 
* Сайт, 
* Рабочие часы по дням недели (в формате словаря "День недели: часы работы"), 
* Долгота, 
* Широта.

In [1]:
import json 
import csv
import pprint
lst=[]
with open('cinema.json', 'r') as f:
    lst.append(json.load(f))

#lst   

In [ ]:
lst_rez=[]
for i in lst:
    for j in i:
        try:
            dicti={}
            dicti['ID'] = j['global_id']
            dicti['CommonName'] = j['CommonName']
            
            str_ad = ''
            for z in j['ObjectAddress']:
                for k1, el1 in z.items():
                    if k1 == 'Address':
                        lst1 = []
                        lst1 = list(el1.split(','))
                        str_ad += str(lst1[0]) + ',' + str(lst1[1])
            dicti['Address'] = str_ad
            
            dicti['Site'] = j['WebSite']
            
            dic_wh = {}
            for m in j['WorkingHours']:
                a = ''
                b = ''
                for k,el in m.items():
                    if k == 'DayWeek':
                        a += el 
                    if k == 'WorkHours':
                        b += el 
                    if a != '' and b!= '':
                        dic_wh[a]=b
                           
            dicti['WorkingHours'] = dic_wh
            
            dicti['Length'] = j['geoData']['coordinates'][0][0]
            dicti['Width'] = j['geoData']['coordinates'][0][1]
           
            lst_rez.append(dicti)
        except TypeError:
            pass
for x in lst_rez:
    print(x)        

In [ ]:
with open('cinema2.json', 'w',encoding = "utf-8") as f:
    json.dump(lst_rez, f,ensure_ascii=False)

In [ ]:
with open('cinema2.csv', 'w', newline = '', encoding = "utf-8") as f2:
    columns = ['ID','CommonName','Address','Site','WorkingHours','Length','Width']
    writer = csv.DictWriter(f2, fieldnames=columns)
    writer.writeheader()
    
    writer.writerows(lst_rez)    

In [ ]:
with open('cinema2.csv', "r", newline="",encoding = "utf-8") as file:
    reader = csv.DictReader(file)
    for row in reader:
        print(row["ID"], "-", row["CommonName"], "-", row["Address"], "-", row["WorkingHours"], "-",row["Site"], "-", row["Length"], "-",row['Width'])
        

2) Присутствуют два csv-файла. В одном ID и координаты узлов графа (points.csv), в другом - координаты ребер графа, полученные с географической карты (polylines.csv). Необходимо к координатам из ребер графа привязать соответствующие им узлы графа (смотреть по близости их координат) и создать csv файл с полями "ID начального узла ребра" и "ID конечного узла ребра". 

**Важно**: координаты узлов могут минимально расходиться с координатами начала и концов ребер, поэтому свести по точному соответствию не получится.

Справка по работе с csv: https://docs.python.org/3/library/csv.html

In [39]:
lst_lines_old = []
with open('polylines.csv', 'r') as file:
    reader = csv.reader(file)
    for row in reader:
        if row != []:
            lst_lines_old.append(row)
pprint.pprint(lst_lines_old)

[['(44.1652,56.1896)',
  '(44.04,56.2379)',
  '(44.0108,56.2366)',
  '(44.0053,56.2371)',
  '(44.0024,56.239)',
  '(44.0034,56.241)',
  '(44.0006,56.2399)',
  '(43.9941,56.2404)',
  '(43.9909,56.2407)',
  '(43.9818,56.2409)',
  '(43.9687,56.244)',
  '(43.9692,56.2455)',
  '(43.9735,56.2474)',
  '(43.9778,56.2455)',
  '(43.9779,56.2472)',
  '(43.9828,56.2465)',
  '(43.9854,56.2442)',
  '(43.9854,56.2452)',
  '(43.9864,56.2459)',
  '(43.9915,56.2443)'],
 ['(43.9865,56.2459)', '(43.9847,56.2491)'],
 ['(44.0054,56.2372)', '(44.0002,56.2338)'],
 ['(43.9734,56.2474)',
  '(43.9677,56.2493)',
  '(43.9685,56.2541)',
  '(43.9732,56.2596)'],
 ['(43.9821,56.2409)',
  '(43.9797,56.2373)',
  '(43.9837,56.2357)',
  '(43.9746,56.2355)',
  '(43.9752,56.2317)',
  '(43.9651,56.2344)',
  '(43.9656,56.235)',
  '(43.9601,56.2385)',
  '(43.955,56.2439)'],
 ['(43.9651,56.2344)',
  '(43.9571,56.2315)',
  '(43.9452,56.2325)',
  '(43.9396,56.2314)'],
 ['(43.9571,56.2316)', '(43.9512,56.2257)'],
 ['(43.9505,56.22

In [40]:
lst_lines_new = []
for i in lst_lines_old:
    k = 0
    start_point = ''
    end_point = ''
    while k <= len(i) - 2:
        start_point = i[k] 
        end_point = i[k+1] 
        k +=1
        lst_lines_new.append([start_point,end_point])
        
pprint.pprint(lst_lines_new)    

[['(44.1652,56.1896)', '(44.04,56.2379)'],
 ['(44.04,56.2379)', '(44.0108,56.2366)'],
 ['(44.0108,56.2366)', '(44.0053,56.2371)'],
 ['(44.0053,56.2371)', '(44.0024,56.239)'],
 ['(44.0024,56.239)', '(44.0034,56.241)'],
 ['(44.0034,56.241)', '(44.0006,56.2399)'],
 ['(44.0006,56.2399)', '(43.9941,56.2404)'],
 ['(43.9941,56.2404)', '(43.9909,56.2407)'],
 ['(43.9909,56.2407)', '(43.9818,56.2409)'],
 ['(43.9818,56.2409)', '(43.9687,56.244)'],
 ['(43.9687,56.244)', '(43.9692,56.2455)'],
 ['(43.9692,56.2455)', '(43.9735,56.2474)'],
 ['(43.9735,56.2474)', '(43.9778,56.2455)'],
 ['(43.9778,56.2455)', '(43.9779,56.2472)'],
 ['(43.9779,56.2472)', '(43.9828,56.2465)'],
 ['(43.9828,56.2465)', '(43.9854,56.2442)'],
 ['(43.9854,56.2442)', '(43.9854,56.2452)'],
 ['(43.9854,56.2452)', '(43.9864,56.2459)'],
 ['(43.9864,56.2459)', '(43.9915,56.2443)'],
 ['(43.9865,56.2459)', '(43.9847,56.2491)'],
 ['(44.0054,56.2372)', '(44.0002,56.2338)'],
 ['(43.9734,56.2474)', '(43.9677,56.2493)'],
 ['(43.9677,56.2493)

In [41]:
lst_points = []
with open('points.csv', 'r') as file:
    reader = csv.reader(file)
    for row in reader:
        if row != []:
            lst_points.append(row)

dict_points = {}
for i, el in enumerate(lst_points, 1):
    dict_points[i] = el
        
            
pprint.pprint(dict_points)

{1: ['43.93784', '56.28922'],
 2: ['44.000857', '56.263747'],
 3: ['43.945512', '56.280012'],
 4: ['44.017494', '56.316729'],
 5: ['44.013892', '56.315785'],
 6: ['44.027977', '56.31892'],
 7: ['44.027914', '56.318251'],
 8: ['43.874796', '56.32532'],
 9: ['44.052061', '56.293911'],
 10: ['44.0473', '56.294106'],
 11: ['44.001234', '56.322594'],
 12: ['44.025561', '56.309344'],
 13: ['44.011188', '56.321246'],
 14: ['44.013847', '56.320338'],
 15: ['43.826413', '56.362237'],
 16: ['44.066425', '56.299056'],
 17: ['44.066775', '56.284469'],
 18: ['43.783231', '56.380009'],
 19: ['44.007262', '56.317467'],
 20: ['43.965185', '56.23438'],
 21: ['43.978489', '56.295095'],
 22: ['44.016757', '56.211053'],
 23: ['43.968814', '56.244015'],
 24: ['43.98307', '56.278348'],
 25: ['44.033699', '56.308051'],
 26: ['43.935774', '56.327081'],
 27: ['44.045692', '56.301713'],
 28: ['43.991604', '56.244215'],
 29: ['44.07363', '56.29506'],
 30: ['43.922254', '56.257818'],
 31: ['43.853003', '56.233719

In [42]:
def get_id(real_point):
    point_id = 0
    for k,v in dict_points.items():
        if real_point == v:
            point_id = k
    return(point_id)

In [60]:
def get_real_point_id(point):
    id_rezult = ''
    point_lat = point.split(',')[0][1:]
    point_lon = point.split(',')[1][:-1]
    
    for i in lst_points:
        if -0.0005 <= float(i[0]) - float(point_lat) <= 0.0005 and -0.0005<= float(i[1])- float(point_lon) <= 0.0005:
            id_rezult = get_id(i)
            break
        else:
            id_rezult = 'None'
    return(id_rezult)

In [75]:
lst_rezult = []
count_error = 0
for lines in lst_lines_new:
    point1 = ''
    point2 = ''
    
    point1 = get_real_point_id(lines[0])
    point2 = get_real_point_id(lines[1])
    if point1 != point2 and point1 != 'None' and point2!= 'None':
        lst_rezult.append({
            'ID начального узла ребра' : point1, 'ID конечного узла ребра': point2
        })
    else:
        count_error +=1

pprint.pprint(lst_rezult)
print(f'count error = {count_error}')

[{'ID конечного узла ребра': 184, 'ID начального узла ребра': 86},
 {'ID конечного узла ребра': 434, 'ID начального узла ребра': 184},
 {'ID конечного узла ребра': 198, 'ID начального узла ребра': 434},
 {'ID конечного узла ребра': 199, 'ID начального узла ребра': 198},
 {'ID конечного узла ребра': 200, 'ID начального узла ребра': 199},
 {'ID конечного узла ребра': 197, 'ID начального узла ребра': 200},
 {'ID конечного узла ребра': 196, 'ID начального узла ребра': 197},
 {'ID конечного узла ребра': 195, 'ID начального узла ребра': 196},
 {'ID конечного узла ребра': 194, 'ID начального узла ребра': 195},
 {'ID конечного узла ребра': 23, 'ID начального узла ребра': 194},
 {'ID конечного узла ребра': 298, 'ID начального узла ребра': 23},
 {'ID конечного узла ребра': 52, 'ID начального узла ребра': 298},
 {'ID конечного узла ребра': 540, 'ID начального узла ребра': 52},
 {'ID конечного узла ребра': 245, 'ID начального узла ребра': 540},
 {'ID конечного узла ребра': 174, 'ID начального узла

In [76]:
with open('points_rezult.csv', 'w', newline ='', encoding = 'UTF-8') as file:
    colums = ['ID начального узла ребра','ID конечного узла ребра']
    writer = csv.DictWriter(file, fieldnames = colums)
    writer.writeheader()
    writer.writerows(lst_rezult)
    

In [72]:
print('3) Создать json-файл, состоящий из ребер графа. Каждое ребро должно иметь уникальный ID (например, номер по порядку), к которому должны быть привязаны данные о начальном узле ("Source"), данные о конечном узле ребра ("Target"). Каждый узел в json должен содержать исходный id, long (долготу) и lat (широту).')

3) Создать json-файл, состоящий из ребер графа. Каждое ребро должно иметь уникальный ID (например, номер по порядку), к которому должны быть привязаны данные о начальном узле ("Source"), данные о конечном узле ребра ("Target"). Каждый узел в json должен содержать исходный id, long (долготу) и lat (широту).


In [78]:
def point_info(point_id):
    dict_return = {}
    for k,v in dict_points.items():
        if k == point_id:
            dict_return['id'] = k
            dict_return['long'] = v[1]
            dict_return['lat'] = v[0]
    return(dict_return)

In [81]:
list_for_json = []
current_id = 1
for i in lst_rezult:
    point_dict = {}
    
    list_for_json.append({
        'ID':current_id, 'Source': point_info(i['ID начального узла ребра']),'Target':  point_info(i['ID конечного узла ребра'])
    })
    current_id +=1

pprint.pprint(list_for_json)

[{'ID': 1,
  'Source': {'id': 86, 'lat': '44.165527', 'long': '56.189729'},
  'Target': {'id': 184, 'lat': '44.040248', 'long': '56.237637'}},
 {'ID': 2,
  'Source': {'id': 184, 'lat': '44.040248', 'long': '56.237637'},
  'Target': {'id': 434, 'lat': '44.010828', 'long': '56.236531'}},
 {'ID': 3,
  'Source': {'id': 434, 'lat': '44.010828', 'long': '56.236531'},
  'Target': {'id': 198, 'lat': '44.005367', 'long': '56.237131'}},
 {'ID': 4,
  'Source': {'id': 198, 'lat': '44.005367', 'long': '56.237131'},
  'Target': {'id': 199, 'lat': '44.002528', 'long': '56.238932'}},
 {'ID': 5,
  'Source': {'id': 199, 'lat': '44.002528', 'long': '56.238932'},
  'Target': {'id': 200, 'lat': '44.003453', 'long': '56.240868'}},
 {'ID': 6,
  'Source': {'id': 200, 'lat': '44.003453', 'long': '56.240868'},
  'Target': {'id': 197, 'lat': '44.00048', 'long': '56.239868'}},
 {'ID': 7,
  'Source': {'id': 197, 'lat': '44.00048', 'long': '56.239868'},
  'Target': {'id': 196, 'lat': '43.994102', 'long': '56.240303

 {'ID': 489,
  'Source': {'id': 456, 'lat': '43.893472', 'long': '56.315311'},
  'Target': {'id': 214, 'lat': '43.898242', 'long': '56.311097'}},
 {'ID': 490,
  'Source': {'id': 214, 'lat': '43.898242', 'long': '56.311097'},
  'Target': {'id': 216, 'lat': '43.905627', 'long': '56.314796'}},
 {'ID': 491,
  'Source': {'id': 216, 'lat': '43.905627', 'long': '56.314796'},
  'Target': {'id': 213, 'lat': '43.909444', 'long': '56.316893'}},
 {'ID': 492,
  'Source': {'id': 213, 'lat': '43.909444', 'long': '56.316893'},
  'Target': {'id': 218, 'lat': '43.918473', 'long': '56.317143'}},
 {'ID': 493,
  'Source': {'id': 218, 'lat': '43.918473', 'long': '56.317143'},
  'Target': {'id': 553, 'lat': '43.918679', 'long': '56.312919'}},
 {'ID': 494,
  'Source': {'id': 553, 'lat': '43.918679', 'long': '56.312919'},
  'Target': {'id': 560, 'lat': '43.915481', 'long': '56.310967'}},
 {'ID': 495,
  'Source': {'id': 214, 'lat': '43.898242', 'long': '56.311097'},
  'Target': {'id': 427, 'lat': '43.889259', '

In [82]:
with open('points_rezult.json', 'w') as file:
    json.dump(list_for_json, file)